In [8]:
import requests
import pandas as pd
import numpy as np
import sddk
import spacy
import zipfile
import io
import re
pd.set_option('display.max_columns', 500)

In [2]:
conf = sddk.configure("SDAM_root", "648597@au.dk")

sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ········
connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/SDAM_root/


In [3]:
PHI = sddk.read_file("SDAM_data/PHI/PHI_text_cleaned_2020-12-16.json", "df", conf)

In [4]:
PHI.head(5)

,URL,Book,Text,hdr1,hdr2,tildeinfo,note,lines,metadata,data,filename,PHI_ID,string_pythia,clean_text_conservative,clean_text_interpretive_word,clean_text_interpretive_sentence,clean_text_pythia
0,/text/1?location=1701&patt=&bookid=4&offset=0&...,IG I³,1,Regions\nAttica (IG I-III),IG I³\n1,Att. — Ath.: Akr. — stoich. 35 — c. 510-500 a....,,12,1\n\n\n\n5\n\n\n\n\n10\n\n,ἔδοχσεν το͂ι δέμοι· τ̣[ὸς ἐ Σ]αλαμ̣[ῖνι κλερόχ...,IG-I³.csv,1,ἔδοχσεν τοι δέμοι τ[ὸς ἐ σ]αλαμ[ῖνι κλερόχ]ος ...,ἔδοχσεν το͂ι δέμοι ταλαμος οἰκε͂ν ἐᾶ Σαλαμῖνι ...,ἔδοχσεν το͂ι δέμοι τὸς ἐ Σαλαμῖνι κλερόχος οἰκ...,ἔδοχσεν το͂ι δέμοι τὸς ἐ Σαλαμῖνι κλερόχος οἰκ...,ἔδοχσεν τοι δέμοι τὸς ἐ σαλαμῖνι κλερόχος οἰκε...
1,/text/2?location=1701&patt=&bookid=4&offset=0&...,IG I³,2,Regions\nAttica (IG I-III),IG I³\n2,Att. — non-stoich. — c. 500 a.,,14,1\n\n\n\n5\n\n\n\n\n10\n\n\n\n,[․․8-9․․․]ν̣ βολ — — — — — — — — — —\n[․6-7․․]...,IG-I³.csv,2,[--------9---]ν βολ ---------- [------7--] α ἑ...,ν βολ α ℎεκον σιον γνοσθε͂ι δὲ ν ἀτεχνος μὲ π ...,ν βολ α ℎεκον σιον γνοσθε͂ι δὲ ν ἀτεχνος μὲ π ...,"ν βολ ․ α ⋮ ℎεκον σιον, γνοσθε͂ι δὲ ν ἀτεχνος ...",ν βολ α ἑκον σιον γνοσθει δὲ ν ἀτεχνος μὲ π ἄλ...
2,/text/3?location=1701&patt=&bookid=4&offset=0&...,IG I³,3,Regions\nAttica (IG I-III),IG I³\n3,Att. — stoich. 21 — 490-480 a.,,13,1\n\n\n\n5\n\n\n\n\n10\n\n\n,[․]αρ[․․․․]ι ℎερακλειο[․․5․․]\n[․]αρ̣ο#⁷[․] τι...,IG-I³.csv,3,[-]αρ[----]ι ἑρακλειο[-----] [-]αρο [-] τιθένα...,αρι ℎερακλειο αρο τιθέναι τὸς ἀέτας τριάκοντα ...,αρι ℎερακλειο αρο τιθέναι τὸς ἀθλοθέτας τριάκο...,αρι ℎερακλειο αρο τιθέναι τὸς ἀθλοθέτας τριάκο...,αρ ι ἑρακλειο αρο τιθέναι τὸς ἀθλοθέτας τριάκο...
3,/text/4?location=1701&patt=&bookid=4&offset=0&...,IG I³,4,Regions\nAttica (IG I-III),IG I³\n4,Att. — stoich. 38 — 485/4 a.,,56,face A.1\n\n\n\n5\n\n\n\n\n10\n\n\n\n\n15\n\n\...,[․․․․․․․․․․․․․․․․․․38․․․․․․․․․․․․․․․․․․]\n[․․․...,IG-I³.csv,4,[--------------------------------------] [----...,δέ τις αν ἒ φρορὰν μ ντέκοντα δχμὰς τ ας ℎες π...,ἐὰν δέ τις αν ἒ φρορὰν μὲ πεντέκοντα δραχμὰς τ...,ἐὰν δέ τις αν ⋮ ἒ φρορὰν ⋮ μὲ πεντέκοντα ⋮ δρα...,ἐὰν δέ τις αν ἒ φρορὰν μὲ πεντέκοντα δραχμὰς τ...
4,/text/5?location=1701&patt=&bookid=4&offset=0&...,IG I³,5,Regions\nAttica (IG I-III),IG I³\n5,Att. — c. 500 a.,,6,1\n\n\n\n5\n,[ἔδοχσε]ν [⋮ τε͂ι βολε͂ι] ⋮ καὶ [τ]ο͂ι δέμοι ⋮...,IG-I³.csv,5,[ἔδοχσε]ν [ τει βολει] καὶ [τ]οι δέμοι ὅτε παρ...,ν καὶ ο͂ι δέμοι ℎότε Παραιβάτες λεια θν τὸς ℎι...,ἔδοχσεν τε͂ι βολε͂ι καὶ το͂ι δέμοι ℎότε Παραιβ...,ἔδοχσεν ⋮ τε͂ι βολε͂ι ⋮ καὶ το͂ι δέμοι ⋮ ℎότε ...,ἔδοχσεν τει βολει καὶ τοι δέμοι ὅτε παραιβάτες...


In [5]:
len(PHI[PHI["clean_text_pythia"] ==  PHI["clean_text_interpretive_word"]])

34206

In [29]:
# sentences
# add column with sentences
v_ud = "’"
v_agt = "ʼ"

def get_sentences(string):
    splits = re.split(r"(\.\s|\·\s|\:\s|\;\s|\?\s|\!\s)" , string)
    sentences = []
    for n in range(0, len(splits), 2):
        try:
            sentence = splits[n]+re.sub("\s$", "", splits[n+1])
            sentence = "".join([c for c in sentence if re.match("[\w|\s|\.|\,|\·|\:|\;|\?|\!]", c)]) 
            sentence = re.sub("^\s*", "", sentence)
            sentence = re.sub(v_ud, v_agt, sentence)
            sentences.append(sentence)
        except:
            sentence = splits[n]
            sentence = re.sub("^\s*", "", sentence)
            if len(sentence) > 0:
                sentences.append(sentence)
            break
    sentences = [sent for sent in sentences if re.search("\w", sent)]
    return sentences

In [30]:
PHI["sents"] = PHI["clean_text_pythia"].apply(get_sentences)

In [31]:
PHI["sents_N"] = PHI["sents"].apply(lambda x: len(x))

In [40]:
PHI["sents_N"].sum()

309734

In [42]:
PHI["clean_text_pythia"].apply(lambda x: len(x.split())).sum()

4419743

In [32]:
len(PHI[PHI["sents_N"] > 1]) # previously: 22321

42973

In [33]:
len(PHI[PHI["sents_N"] > 2]) # 11125

15617

In [34]:
# look at some inscriptions having more than two sentences.
PHI[PHI["sents_N"] > 2]["sents"].tolist()[:5]

[['δραχμεισι τες τὸς ιο μενος δεμο τον πόλεον δοκει ἀνατιθ α ἐάν τις τον ον ἒ ὁσαλ ο ἒ εν ορ νας ἵνα αντο ιν σ τον πόλεον ταυτ χρε μεμβολ εχθεο χε ν δε ν λο ἐὰν δὲ μέ ἑκαστ ο σε α ιν ιμ ιας πε ετα τεν ια ο λ πο σειαν λ λαμ ολει καὶ μὲ νεοτερ χρέσθο τοι ἱεροι ἐὰν δὲ ι μὲ χρέσθο ἐὰν δὲ ἰ κατὰ ταὐτὰ ταῦτα ἐὰν πλεῖστον κατὰ τὲν δύναμιν πρᾶχσαι δ ἔκπραχσιν ἐὰν δὲ μὲ ἐγδοι τὸν ὀφλόντα μὲ χρέσθο τοι ἱεροι ἐὰν ἀμφισβετοσι μὲ κλεθεναι ἐμ πόλει εν ἐλθοσαν ἀδικ ὕστερον ἑ βολὲ α ι τὸν ἀθεναῖον μὲ ἐκ γες πο τούτον τον πόλεον μεδὲ ἁμος βιᾶσθαι ἐὰν μὲ δίκεν ὀφλόντα ἐπιχορίαν ἒ ἐς πολεμίος λεφθέντα ἕτις δ ἂν τομ πόλεον μὲ ἐθέλει δίκας διδόναι καὶ δέχεσθαι ἀθεναίοισιν ἀπὸ χσυμβολον.',
  'ια ντο τος τὰ μὲν ἁκόσια ἁπλει τὰ δὲ ἡ εκόσια διπλει σπονδὰς εἶναι τοῖσι μύστεσιν καὶ τοῖς ἐπόπτεισιν καὶ τοῖς ἀκολούθοισιν καὶ χρέμασιν τον ὀθνείον καὶ ἀθεναῖοισιν ἅπασιν ἄρχεν δὲ τὸν χρόνον τον σπονδον το μεταγειτνιονος μενὸς ἀπὸ διχομενίας καὶ τὸν βοεδρομιονα καὶ το υανοφσιονος μέχρι δεκάτες ἱσταμένο τὰς δὲ σπονδὰς 

# Download spacy model from sciencedata, load it etc.

In [35]:
# download our spacy model and unzip it.
#!curl https://sciencedata.dk//shared/b8f6c317c433385ce79a9531a0fbf658?download -o ../data/spacy_grc_model.zip
#!unzip  -: ../data/spacy_grc_model.zip

In [36]:
nlp = spacy.load("../models/spacy_grc_model_7")
path = "../models/spacy_grc_model_7/grc_lemmatization.py"
exec(open(path).read())
nlp.add_pipe(grc_doc_lemmatizer, "grc_doc_lemmatizer", after="tagger")

In [37]:
def apply_nlp(sentences_list):
    spacy_docs = []
    for sentence in sentences_list:
        doc = nlp(sentence)
        spacy_docs.append(doc)
    return spacy_docs

In [43]:
%%time
def try_apply_nlp(sents):
    try: return apply_nlp(sents)
    except: return None

## apply it on the normalized string
PHI_docs_series = PHI["sents"].apply(try_apply_nlp)

CPU times: user 16min 7s, sys: 2.62 s, total: 16min 10s
Wall time: 16min 13s


In [44]:
def extract_tagged(sentences):
    tagged_sentences = []
    try:
        for sent in sentences:
            tokens = []
            for token in sent:
                tokens.append((token.text, token.pos_,token.lemma_))
            tagged_sentences.append(tokens)
    except: pass
    return tagged_sentences

In [45]:
extract_tagged(PHI_docs_series.iloc[0])

[[('ἔδοχσεν', 'VERB', 'ἔδοχσεν'),
  ('τοι', 'PRON', 'σύ'),
  ('δέμοι', 'ADJ', 'δέμοι'),
  ('τὸς', 'PRON', 'τὸς'),
  ('ἐ', 'INTJ', 'ἒ'),
  ('σαλαμῖνι', 'NOUN', 'Σαλαμίς'),
  ('κλερόχος', 'NOUN', 'κλερόχος'),
  ('οἰκεν', 'VERB', 'οἰκεν'),
  ('ἐᾶ', 'INTJ', 'ἔα'),
  ('σαλαμῖνι', 'NOUN', 'Σαλαμίς'),
  ('λεν', 'VERB', 'λεν'),
  ('ἀθένεσι', 'NOUN', 'ἀθένεσι'),
  ('τελεν', 'VERB', 'τελεν'),
  ('καὶ', 'CCONJ', 'καί'),
  ('στρατεύεσθαι', 'VERB', 'στρατεύω'),
  ('τὰ', 'PRON', 'ὁ'),
  ('δ', 'PRON', 'δ'),
  ('ἐ', 'INTJ', 'ἒ'),
  ('σαλαμῖνι', 'NOUN', 'Σαλαμίς'),
  ('μὲ', 'PRON', 'ἐγώ'),
  ('μισθον', 'NOUN', 'μισθός'),
  ('ἐὰ', 'INTJ', 'ἔα'),
  ('μὲ', 'PRON', 'ἐγώ'),
  ('οἰκ', 'ADV', 'οἰκ'),
  ('ο', 'ADV', 'ο'),
  ('μισθόμενο', 'VERB', 'μισθόμενο'),
  ('ἐὰν', 'SCONJ', 'ἐάν'),
  ('δὲ', 'ADV', 'δέ'),
  ('μισθοι', 'NOUN', 'μισθός'),
  ('ἀποτίνεν', 'VERB', 'ἀποτίνεν'),
  ('τὸ', 'DET', 'ὁ'),
  ('μισθόμενον', 'VERB', 'μισθόμενον'),
  ('καὶ', 'CCONJ', 'καί'),
  ('τὸ', 'DET', 'ὁ'),
  ('μισθοντα', 'VERB', 'μι

In [46]:
def extraxt_lem_sents(sentences):
    lemmatized_sentences = []
    try:
        for sent in sentences:
            lemmata = []
            for token in sent:
                if token.pos_ in ["NOUN", "PROPN", "ADJ", "VERB"]:
                    lemmata.append(token.lemma_)
            lemmatized_sentences.append(lemmata)
    except: pass
    return lemmatized_sentences

In [47]:
%%time
lem_sents_series = PHI_docs_series.apply(extraxt_lem_sents)

CPU times: user 4.91 s, sys: 100 ms, total: 5.01 s
Wall time: 5.04 s


In [48]:
%%time
PHI["lem_sents"] = lem_sents_series
#PHI["lemmata"] = PHI["string_normalized"].apply(lambda x: lemmatize_string(x, all_lemmata=False, filter_by_postag=["n", "a", "v"], involve_unknown=True))

CPU times: user 3.91 ms, sys: 0 ns, total: 3.91 ms
Wall time: 3.72 ms


In [49]:
def flat_lemmata(sentences):
    lemmata = [word for sent in sentences for word in sent]
    return lemmata
PHI["lemmata"] = PHI["lem_sents"].apply(flat_lemmata)

In [51]:
PHI["lemmata"].apply(lambda x: len(x)).sum()

2805617

In [50]:
sddk.write_file("SDAM_data/PHI/PHI_lemmatized_20201217.json", PHI, conf)

Your <class 'pandas.core.frame.DataFrame'> object has been succefully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/PHI/PHI_lemmatized_20201217.json"


# Reading the data back

In [4]:
PHI = sddk.read_file("SDAM_data/PHI/PHI_dated_lemmatized_20201214.json", "df", conf)

In [6]:
def count_words(x):
    try:
        return len(x.split())
    except:
        return 0
PHI["clean_text_pythia_ext"].apply(count_words).sum() # previously 4469462, then 4507247

11551090

In [10]:
PHI["clean_text_interpretive_word"].apply(count_words).sum()

4507247

In [9]:
PHI["clean_text_pythia"].apply(count_words).sum() # previously 4469462, then 4507247

8704432

In [45]:
PHI["lemmata"].apply(lambda x: len(x)).sum() # previously 2743932, 2846806, 2867815

7132407